In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

def cal_ArrayGC(X, N_x):
    Array_GC = np.zeros((N_x,N_x))
    for i in range(N_x):
        for j in range(N_x):
            if i == j:
                continue
            targetI = 3*i
            targetJ = 3*j+1

            x = X[:,[targetI,targetJ]]
            r = grangercausalitytests(x, maxlag=[1], verbose=False)
            Array_GC[i,j] = -np.log10(r[1][0]['ssr_ftest'][1])

    return Array_GC

In [ ]:
from pyinform.transferentropy import transfer_entropy
import pyinform.utils as utils

def cal_ArrayTE(X, N_x):
    Array_TE = np.zeros((N_x,N_x))

    for i in range(N_x):
        for j in range(N_x):
            if i == j:
                continue
            targetI = 3*i
            targetJ = 3*j+1

            x1,_ = utils.coalesce_series(utils.bin_series(X[:,targetI], b=4)[0])
            x2,_ = utils.coalesce_series(utils.bin_series(X[:,targetJ], b=4)[0])

            r = transfer_entropy(x2, x1, k=5)
            Array_TE[i,j] = r
        
    return Array_TE

In [ ]:
import skccm as ccm
from skccm.utilities import train_test_split

def cal_ArrayCCM(X, N_x):
    Array_CCM = np.zeros((N_x,N_x))

    lag = 1
    embed = 2

    for i in range(N_x):
        for j in range(N_x):
            if i == j:
                continue
            targetI = 3*i
            targetJ = 3*j+1

            xi = X[:,targetI]
            xj = X[:,targetJ]

            ei = ccm.Embed(xi)
            ej = ccm.Embed(xj)
            Xi = ei.embed_vectors_1d(lag,embed)
            Xj = ej.embed_vectors_1d(lag,embed)

            #split the embedded time series
            xitr, xite, xjtr, xjte = train_test_split(Xi,Xj, percent=.75)

            CCM = ccm.CCM() #initiate the class

            #library lengths to test
            len_tr = len(xitr)
            lib_lens = np.arange(10, len_tr, len_tr/50, dtype='int')

            #test causation
            CCM.fit(xitr,xjtr)
            xip, xjp = CCM.predict(xite, xjte,lib_lengths=lib_lens)

            _,scji = CCM.score()

            Array_CCM[i,j] = scji[-1]
    return Array_CCM